# Text datasets

In [ ]:
#| default_exp text.datasets

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export

# torch
import torch
from torch.optim import SGD
import torchtext; torchtext.disable_torchtext_deprecation_warning()
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import vocab as torchtext_vocab
from torch.utils.data import DataLoader, dataset, Dataset, random_split

# L
from lightning import LightningDataModule

# hf
import datasets
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, DataCollatorForLanguageModeling

# data 
import pandas as pd
import numpy as np

# ui
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

# param
from omegaconf import DictConfig, OmegaConf
from hydra.utils import instantiate

# python
from typing import Dict, List, Tuple, Optional, Set, Any
from collections import Counter, OrderedDict
from plum import dispatch
import random
import os
from pathlib import Path
import requests
import re

# nimrod
# from nimrod.models.lm import Vocab
from nimrod.utils import set_seed
from nimrod.data.core import DataModule

import logging
logger = logging.getLogger(__name__)


In [ ]:
SEED = 42
set_seed(SEED)

Seed set to 42


## Vocab
Each row is a list of words (sentence). For each row, extract unique character and add to vocabulary. deals with special characters too.

In [ ]:
#| export
class Vocab:
    def __init__(self,
                data_path: str | os.PathLike='../data/text/tiny_shakespeare.txt', # path to text data file
                specials=['<pad>', '<unk>', '<bos>', '<eos>'], # encode special characters
                ):

        self.data_path = Path(data_path)
        if not self.data_path.exists():
            self._download_data()

        logger.info(f"Vocab: read text file")
        with open(self.data_path, 'r') as f:
            text = f.read()

        chars = set(text)
        if specials is not None:
            for special in specials:
                chars.add(special)

        self._stoi = {c: i for i, c in enumerate(chars)}
        self._itos = {i: c for i, c in enumerate(chars)}
        self.voc = chars


    
    def _download_data(self):
        logger.info(f"Vocab: download data from url")
        url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
        response = requests.get(url)
        response.raise_for_status()
        
        with open(self.data_path, 'w') as f:
            f.write(response.text)

    @dispatch
    def stoi(self, token:str)->int:
        if token not in self._stoi:
            return self._stoi['<unk>']
        return self._stoi[token]

    @dispatch
    # for list of characters
    def stoi(self, tokens:List[str])->List[int]:
        # TODO: deal with unknown tokens
        return [self._stoi[tok] if tok in self._stoi else self._stoi['<unk>'] for tok in tokens]
    
    # @dispatch #TODO
    # def stoi(self, tokens:List[List[str]])->List[List[int]]:
    #     return [self._stoi[u] for tok in tokens for ]
    # TODO:
    # support torch tensors

    @dispatch    
    def itos(self, index:int)->str:
        return self._itos[index]
    
    @dispatch    
    def itos(self, indices:List[int])->List[str]:
        return [self._itos[index] for index in indices]
        
    def __len__(self):
        return len(self.voc)
    
    @property
    def vocabulary(self)->Set:
        return sorted(set([k for k,v in self._stoi.items()]))


### Usage
read text file into a pandas data framew with each row as a new line

In [ ]:
v = Vocab('../data/text/tiny_shakespeare.txt', specials=['<pad>', '<unk>', '<bos>', '<eos>'])
print(v.vocabulary)

[12/19/24 10:00:51] INFO     2024-12-19 10:00:51,988 - INFO - Loading data from                    ]8;id=234053;file:///var/folders/b5/v9y3kpzs29g41d99xvrdp3yr0000gn/T/ipykernel_60811/3273953393.py\3273953393.py]8;;\:]8;id=146316;file:///var/folders/b5/v9y3kpzs29g41d99xvrdp3yr0000gn/T/ipykernel_60811/3273953393.py#12\12]8;;\
                             ../data/text/tiny_shakespeare.txt                                                     

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '<bos>', '<eos>', '<pad>', '<unk>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [ ]:
# egs where token * is not in vocab
print(v.stoi('*'))
print(v.itos(61))

61
<unk>


In [ ]:
print(v.vocabulary)
s = v.stoi(["<bos>","h", "e", "l", "l", "o", "*", "<eos>"])
print(s)
print(v.itos(s))

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '<bos>', '<eos>', '<pad>', '<unk>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
[28, 54, 14, 11, 11, 18, 61, 62]
['<bos>', 'h', 'e', 'l', 'l', 'o', '<unk>', '<eos>']


## Tiny shakespeare

### Char Dataset
C.f. https://karpathy.github.io/char-rnn/ text is a long continuous string

In [ ]:
#| export 

class CharDataset(Dataset):
    def __init__(self,
                data_path: str | os.PathLike='../data/text/tiny_shakespeare.txt', # path to the data file
                context_length: int=3, # context length
                specials=['<pad>', '<unk>', '<bos>', '<eos>'], # encode special characters
                add_sentence_tokens: bool = True, # add special tokens to the data
                ):
        logger.info(f"CharDataset: init")
        
        # vocab will download data if not found
        self.v = Vocab(data_path=data_path, specials=specials)
        self._vocab_size = len(self.v)
        self.context_length = context_length
        self.special_token_pattern = re.compile(f'({re.escape("<bos>")}|{re.escape("<eos>")})')

        with open(data_path, 'r') as f:
            text = f.read()

        if add_sentence_tokens:
            # Split into sentences (roughly, using periods) and add special tokens
            sentences = [s.strip() for s in text.split('.') if s.strip()]
            sentences = ['<bos>' + s + '<eos>' for s in sentences]
            # Join list of words into single continuous text
            text = " ".join(sentences)

        # text = text.replace("\n", " ")
        tokens = self._tokenizer(text)
        self.data = torch.tensor(self.v.stoi(tokens), dtype=torch.long)

    def __len__(self) -> int:
        return len(self.data)

    def _tokenizer(self, text: str) -> List[str]:
        parts = self.special_token_pattern.split(text)
        tokens = []
        for part in parts:
            if part in ["<bos>", "<eos>"]:
                tokens.append(part)
            else:
                tokens.extend(part)
        return tokens

    @property
    def vocabulary(self)->Set:
        return sorted(set([k for k,v in self.v._stoi.items()]))
    
    @property
    def vocab_size(self)->int:
        return self._vocab_size

    @property
    def vocab_class(self)->Vocab:
        return self.v

    def __getitem__(self, i: int) -> Tuple[torch.Tensor, torch.Tensor]:
        # i = random.randint(0, len(self.data) - (self.context_length + 1))
        max_index = len(self.data) - (self.context_length + 1)
        if i > max_index:
            # wrap around to the beginning if we hit the end
            i = i % (max_index + 1)
        chunk = self.data[i : i + self.context_length + 1]
        x = chunk[:-1]
        y = chunk[1:]
        return x, y

    def to_tokens(self, message: str) -> torch.Tensor:
        return torch.tensor([self.v.stoi(s) for s in message], dtype=torch.long)

    def from_tokens(self, tokens: torch.Tensor) -> str:
        return "".join([self.v.itos(int(i)) for i in tokens])

#### Usage

In [ ]:
%%time 
block_size = 3 #context_length
ds = CharDataset(data_path='../data/text/tiny_shakespeare.txt', context_length=block_size, specials=['<pad>', '<unk>', '<bos>', '<eos>'], add_sentence_tokens=True)
# just encode <unk> in case unknown characters are encountered in test set
ds = CharDataset(data_path='../data/text/tiny_shakespeare.txt', context_length=block_size, specials=['<unk>', '<pad>'], add_sentence_tokens=False)
print("vocab size: ", ds.vocab_size)
print(len(ds))
for i in range(2):
    x, y = ds[i]
    print("x:", x,  "itos: ", ds.from_tokens(x), "\ny:", y, "itos: ", ds.from_tokens(y)[-1])


[12/19/24 10:01:17] INFO     2024-12-19 10:01:17,528 - INFO - CharDataset:                         ]8;id=101414;file:///var/folders/b5/v9y3kpzs29g41d99xvrdp3yr0000gn/T/ipykernel_60811/2462447885.py\2462447885.py]8;;\:]8;id=376417;file:///var/folders/b5/v9y3kpzs29g41d99xvrdp3yr0000gn/T/ipykernel_60811/2462447885.py#10\10]8;;\
                             ../data/text/tiny_shakespeare.txt, context_length: 3, specials:                       
                             ['<pad>', '<unk>', '<bos>', '<eos>'], add_sentence_tokens: True                       

                    INFO     2024-12-19 10:01:17,529 - INFO - Loading data from                    ]8;id=846335;file:///var/folders/b5/v9y3kpzs29g41d99xvrdp3yr0000gn/T/ipykernel_60811/3273953393.py\3273953393.py]8;;\:]8;id=45561;file:///var/folders/b5/v9y3kpzs29g41d99xvrdp3yr0000gn/T/ipykernel_60811/3273953393.py#12\12]8;;\
                             ../data/text/tiny_shakespeare.txt                                                     

                    INFO     2024-12-19 10:01:17,616 - INFO - CharDataset:                         ]8;id=396922;file:///var/folders/b5/v9y3kpzs29g41d99xvrdp3yr0000gn/T/ipykernel_60811/2462447885.py\2462447885.py]8;;\:]8;id=82627;file:///var/folders/b5/v9y3kpzs29g41d99xvrdp3yr0000gn/T/ipykernel_60811/2462447885.py#10\10]8;;\
                             ../data/text/tiny_shakespeare.txt, context_length: 3, specials:                       
                             ['<unk>', '<pad>'], add_sentence_tokens: False                                        

                    INFO     2024-12-19 10:01:17,617 - INFO - Loading data from                    ]8;id=648564;file:///var/folders/b5/v9y3kpzs29g41d99xvrdp3yr0000gn/T/ipykernel_60811/3273953393.py\3273953393.py]8;;\:]8;id=928463;file:///var/folders/b5/v9y3kpzs29g41d99xvrdp3yr0000gn/T/ipykernel_60811/3273953393.py#12\12]8;;\
                             ../data/text/tiny_shakespeare.txt                                                     

vocab size:  67
1115394
x: tensor([ 6, 37, 17]) itos:  Fir 
y: tensor([37, 17, 29]) itos:  s
x: tensor([37, 17, 29]) itos:  irs 
y: tensor([17, 29, 35]) itos:  t
CPU times: user 159 ms, sys: 9.46 ms, total: 168 ms
Wall time: 168 ms


In [ ]:
x,y = ds[0]
print("x:", x,  "itos: ", ds.from_tokens(x), "\ny:", y, "itos: ", ds.from_tokens(y))
print("vocab size: ", ds.vocab_size)
print("vocabulary: ", ds.vocabulary)

x: tensor([ 6, 37, 17]) itos:  Fir 
y: tensor([37, 17, 29]) itos:  irs
vocab size:  67
vocabulary:  ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '<pad>', '<unk>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [ ]:
print(len(ds))
t = len(ds)*torch.tensor((0.8, 0.1, 0.1))
lengths = [int(p * len(ds)) for p in (0.8, 0.1, 0.1)]
lengths[-1] = len(ds) - sum(lengths[:-1])
print(lengths)

random_split(ds, lengths)

1118332
[894665, 111833, 111834]


[<torch.utils.data.dataset.Subset>,
 <torch.utils.data.dataset.Subset>,
 <torch.utils.data.dataset.Subset>]

### Char Data Module

In [ ]:
#| export

class CharDataModule(DataModule, LightningDataModule):
    def __init__(self,
            # dataset
            data_path: str | os.PathLike = '../data/text/tiny_shakespeare.txt',
            specials=['<pad>', '<unk>', '<bos>', '<eos>'],
            add_sentence_tokens: bool = False,
            # data module
            train_val_test_split: Tuple[int, int, int] = (0.8, 0.1, 0.1),
            context_size: int = 3,
            batch_size: int = 32,
            num_workers: int = 1,
            pin_memory: bool = False,
            persistent_workers: bool = False,
            random_split: bool = True
            ):

        logger.info(f"CharDataModule: init")

        super().__init__(
            train_val_test_split=train_val_test_split,
            batch_size=batch_size,
            num_workers=num_workers,
            pin_memory=pin_memory,
            persistent_workers=persistent_workers,
            )
        self.save_hyperparameters()
        self.ds: CharDataset = None
    
    def prepare_data(self) -> None:
        pass

    
    def setup(self, stage: Optional[str] = None) -> None:
        logger.info("CharDataModule: setup, split datasets")
        # run in each GPU process. define, split DS, etc.
        self.ds = CharDataset(
            self.hparams.data_path,
            self.hparams.context_size,
            self.hparams.specials,
            self.hparams.add_sentence_tokens,
            )
        if self.hparams.random_split:
            lengths = [int(p * len(self.ds)) for p in self.hparams.train_val_test_split]
            lengths[-1] = len(self.ds) - sum(lengths[:-1])
            self.data_train, self.data_val, self.data_test = random_split(self.ds, lengths)
        else:
            self.data_train, self.data_val, self.data_test = self._sequential_split(self.ds, self.hparams.train_val_test_split)
    
    @property
    def vocab_size(self)->int:
        if self.ds is None:
            raise ValueError("Dataset not initialized")
        return self.ds.vocab_size



#### Usage

In [ ]:
dm = CharDataModule(
    data_path="../data/text/tiny_shakespeare.txt",
    add_sentence_tokens=False,
    specials=['<unk>', '<pad>'],
    context_size=3,
    train_val_test_split = (0.8, 0.1, 0.1),
    random_split=False,
    batch_size=64,
    num_workers=0,
    pin_memory=False,
    persistent_workers=False,
    )
dm.setup()

[12/19/24 10:04:36] INFO     2024-12-19 10:04:36,080 - INFO - Initalizing CharDataModule           ]8;id=240174;file:///var/folders/b5/v9y3kpzs29g41d99xvrdp3yr0000gn/T/ipykernel_60811/2539470188.py\2539470188.py]8;;\:]8;id=861722;file:///var/folders/b5/v9y3kpzs29g41d99xvrdp3yr0000gn/T/ipykernel_60811/2539470188.py#19\19]8;;\

                    INFO     2024-12-19 10:04:36,081 - INFO - split data                           ]8;id=69403;file:///var/folders/b5/v9y3kpzs29g41d99xvrdp3yr0000gn/T/ipykernel_60811/2539470188.py\2539470188.py]8;;\:]8;id=221231;file:///var/folders/b5/v9y3kpzs29g41d99xvrdp3yr0000gn/T/ipykernel_60811/2539470188.py#35\35]8;;\

                    INFO     2024-12-19 10:04:36,082 - INFO - CharDataset:                         ]8;id=752787;file:///var/folders/b5/v9y3kpzs29g41d99xvrdp3yr0000gn/T/ipykernel_60811/2462447885.py\2462447885.py]8;;\:]8;id=329963;file:///var/folders/b5/v9y3kpzs29g41d99xvrdp3yr0000gn/T/ipykernel_60811/2462447885.py#10\10]8;;\
                             ../data/text/tiny_shakespeare.txt, context_length: 3, specials:                       
                             ['<unk>', '<pad>'], add_sentence_tokens: False                                        

                    INFO     2024-12-19 10:04:36,084 - INFO - Loading data from                    ]8;id=927657;file:///var/folders/b5/v9y3kpzs29g41d99xvrdp3yr0000gn/T/ipykernel_60811/3273953393.py\3273953393.py]8;;\:]8;id=958972;file:///var/folders/b5/v9y3kpzs29g41d99xvrdp3yr0000gn/T/ipykernel_60811/3273953393.py#12\12]8;;\
                             ../data/text/tiny_shakespeare.txt                                                     

In [ ]:
X, Y = dm.data_train[0]
print(dm.ds.from_tokens(X), dm.ds.from_tokens(Y), dm.vocab_size)

Fir irs 67


In [ ]:
len(dm.data_train), len(dm.data_val), len(dm.data_test)
print(dm.data_test[0])
print(len(dm.test_dataloader()))

(tensor([46, 20, 36]), tensor([20, 36, 36]))
1748


In [ ]:
test_dl = dm.test_dataloader()
X,Y = next(iter(test_dl))
print("X (B,T): ", X.shape, "X: ", X[0], "chars: ", dm.ds.from_tokens(X[0]))
print( "Y (B): ", Y.shape, "Y: ", Y[0], "chars: ", dm.ds.from_tokens(Y[0]))

X (B,T):  torch.Size([64, 3]) X:  tensor([46, 20, 36]) chars:  tuf
Y (B):  torch.Size([64, 3]) Y:  tensor([20, 36, 36]) chars:  uff


#### Init from config file

In [ ]:
cfg = OmegaConf.load('../config/text/data/tinyshakespeare.yaml')
print(cfg)
dm = instantiate(cfg)
dm.setup()

{'_target_': 'nimrod.text.datasets.CharDataModule', 'data_path': '../data/text/tiny_shakespeare.txt', 'train_val_test_split': [0.8, 0.1, 0.1], 'batch_size': 64, 'context_size': 3, 'num_workers': 0, 'pin_memory': False, 'persistent_workers': False}


In [ ]:
test_dl = dm.test_dataloader()
X,Y = next(iter(test_dl))
print("X (B,T): ", X.shape, "X: ", X[0], "chars: ", dm.ds.from_tokens(X[0]))
print( "Y (B): ", Y.shape, "Y: ", Y[0], "chars: ", dm.ds.from_tokens(Y[0]))

X (B,T):  torch.Size([64, 20]) X:  tensor([12,  6,  4, 28,  5,  5, 13,  4, 16,  6,  6,  8, 17,  6, 22,  4,  8, 12,
        11,  8]) chars:  he poor beetle, that
Y (B):  torch.Size([64, 20]) Y:  tensor([ 6,  4, 28,  5,  5, 13,  4, 16,  6,  6,  8, 17,  6, 22,  4,  8, 12, 11,
         8,  4]) chars:  e poor beetle, that 


### Hugging Face
https://huggingface.co/learn/nlp-course/chapter7/6?fw=pt

#### Load text file

In [ ]:
dataset = load_dataset("text", data_files="../data/text/tiny_shakespeare.txt") #, split=['train','dev','test'])
print(dataset)
full = dataset['train']

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 40000
    })
})


In [ ]:
train_test = full.train_test_split(train_size=0.8)
test_valid = train_test['test'].train_test_split(train_size=0.5)
shake = DatasetDict({
    'train': train_test['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})
print(shake)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 32000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 4000
    })
    valid: Dataset({
        features: ['text'],
        num_rows: 4000
    })
})


In [ ]:
shake['test'][0]

{'text': "There's no more to be said, but he is banish'd,"}

#### Tokenization / Numericalization

##### Tokenize single element

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')

print("vocab size: ", len(tokenizer))
print("text row 0: ", shake['test'][0]['text'])
tokens = tokenizer.tokenize(shake['test'][0]['text'])
print("tokens of row 0: ", tokens)

context_length = 10
padded = tokenizer(shake['test'][0]['text'], max_length=context_length, truncation=True, return_length=True, return_overflowing_tokens=True)
print("context block & padding for lm: ", padded)
# print(padded.keys())
print('decode single input_id: ', tokenizer.decode(849))
print([tokenizer.decode(x) for x in padded['input_ids']])

vocab size:  50257
text row 0:  There's no more to be said, but he is banish'd,
tokens of row 0:  ['There', "'s", 'Ġno', 'Ġmore', 'Ġto', 'Ġbe', 'Ġsaid', ',', 'Ġbut', 'Ġhe', 'Ġis', 'Ġban', 'ish', "'d", ',']
context block & padding for lm:  {'input_ids': [[1858, 338, 645, 517, 284, 307, 531, 11, 475, 339], [318, 3958, 680, 1549, 11]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1]], 'length': [10, 5], 'overflow_to_sample_mapping': [0, 0]}
decode single input_id:  oth
["There's no more to be said, but he", " is banish'd,"]


/Users/slegroux/miniforge3/envs/nimrod/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


##### Tokenize whole dataset using map

In [ ]:
from omegaconf import OmegaConf

In [ ]:
cfg = {
    "context_length": 10,
    "truncation": True,
    "return_length": True,
    "return_overflowing_tokens": True,
}

cfg = OmegaConf.create(cfg)

# tokenizer function called via dataset map
def tokenize_function(examples:List[dict[str,str]], cfg:OmegaConf=cfg) -> dict[str, List[List[int]]]:
    result = tokenizer(
        examples["text"],
        max_length=cfg.context_length,
        truncation=cfg.truncation,
        return_length=cfg.return_length,
        return_overflowing_tokens=cfg.return_overflowing_tokens
        )
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

In [ ]:
tokenize_function(shake['test'][0])

{'input_ids': [[1858, 338, 645, 517, 284, 307, 531, 11, 475, 339], [318, 3958, 680, 1549, 11]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1]], 'length': [10, 5], 'overflow_to_sample_mapping': [0, 0], 'word_ids': [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [10, 11, 11, 12, 13]]}

In [ ]:
shake_toked = shake.map(
    tokenize_function, batched=True,
    remove_columns=["text"],
    num_proc = 1
)

In [ ]:
print(shake_toked['test'][0])
print([tokenizer.decode(x) for x in shake_toked['test'][0]['input_ids']])
print(tokenizer.decode(shake_toked['test'][0]['input_ids']))

{'input_ids': [1858, 338, 645, 517, 284, 307, 531, 11, 475, 339], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'length': 10, 'overflow_to_sample_mapping': 0, 'word_ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}
['There', "'s", ' no', ' more', ' to', ' be', ' said', ',', ' but', ' he']
There's no more to be said, but he


In [ ]:
for split, dset in shake_toked.items():
    print(split, dset)
    arr_len = np.sum(dset['length'], dtype=np.uint64)


train Dataset({
    features: ['input_ids', 'attention_mask', 'length', 'overflow_to_sample_mapping', 'word_ids'],
    num_rows: 42263
})
test Dataset({
    features: ['input_ids', 'attention_mask', 'length', 'overflow_to_sample_mapping', 'word_ids'],
    num_rows: 5276
})
valid Dataset({
    features: ['input_ids', 'attention_mask', 'length', 'overflow_to_sample_mapping', 'word_ids'],
    num_rows: 5295
})


#### Data Collator

In [ ]:
print(tokenizer.pad_token, tokenizer.eos_token)


None <|endoftext|>


In [ ]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
out = data_collator([shake_toked["test"]['input_ids'][i] for i in range(5)])
# out = default_data_collator(shake_toked['test']['input_ids'][0])
print(out)
# for key in out:
#     print(f"{key} shape: {out[key].shape}")
# print('inputs: ', out['input_ids'])
# print('labels: ', out['labels'])

# data_collator = DefaultDataCollator(tokenizer)
# out = data_collator([shake_toked["test"][i] for i in range(5)])
# print(out)



{'input_ids': tensor([[ 1858,   338,   645,   517,   284,   307,   531,    11,   475,   339],
        [  318,  3958,   680,  1549,    11, 50256, 50256, 50256, 50256, 50256],
        [   40,  8406,   606,   326,   750,  1842,   511,  1499,   338,   922],
        [   45, 12321,    11, 35695,   502,   284, 11906, 10846,   290, 37769],
        [   13,   314, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256]]), 'labels': tensor([[ 1858,   338,   645,   517,   284,   307,   531,    11,   475,   339],
        [  318,  3958,   680,  1549,    11,  -100,  -100,  -100,  -100,  -100],
        [   40,  8406,   606,   326,   750,  1842,   511,  1499,   338,   922],
        [   45, 12321,    11, 35695,   502,   284, 11906, 10846,   290, 37769],
        [   13,   314,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100]])}


In [ ]:
def my_collate(examples, block_size: int, **kwargs):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
example = shake['test'][0]
# concatenated_examples = {k: sum(example[k], []) for k in example.keys()}
print([example[k] for k in example.keys()])

["There's no more to be said, but he is banish'd,"]


In [ ]:
out = data_collator([shake_toked['test']['input_ids'][i] for i in range(4)])
print(out)

{'input_ids': tensor([[ 1858,   338,   645,   517,   284,   307,   531,    11,   475,   339],
        [  318,  3958,   680,  1549,    11, 50256, 50256, 50256, 50256, 50256],
        [   40,  8406,   606,   326,   750,  1842,   511,  1499,   338,   922],
        [   45, 12321,    11, 35695,   502,   284, 11906, 10846,   290, 37769]]), 'labels': tensor([[ 1858,   338,   645,   517,   284,   307,   531,    11,   475,   339],
        [  318,  3958,   680,  1549,    11,  -100,  -100,  -100,  -100,  -100],
        [   40,  8406,   606,   326,   750,  1842,   511,  1499,   338,   922],
        [   45, 12321,    11, 35695,   502,   284, 11906, 10846,   290, 37769]])}


In [ ]:
for i in range(4):
    print([tokenizer.decode(x) for x in out['input_ids'][i]])


['There', "'s", ' no', ' more', ' to', ' be', ' said', ',', ' but', ' he']
[' is', ' ban', 'ish', "'d", ',', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>']
['I', ' bid', ' them', ' that', ' did', ' love', ' their', ' country', "'s", ' good']
['N', 'urse', ',', ' commend', ' me', ' to', ' thy', ' lady', ' and', ' mistress']


#### Dataloader

In [ ]:
test_dl = DataLoader(
    shake_toked['test']['input_ids'],
    batch_size=128,
    collate_fn=data_collator,
    num_workers=0,
)

In [ ]:
#!head ../data/text/tiny_shakespeare.txt

In [ ]:
b = next(iter(test_dl))
print(b['input_ids'].shape)
print(b['input_ids'][1])
print(b['labels'][1])
# for i in range(128):
#     print([tokenizer.decode(x) for x in b['input_ids'][i]])

torch.Size([128, 10])
tensor([  318,  3958,   680,  1549,    11, 50256, 50256, 50256, 50256, 50256])
tensor([ 318, 3958,  680, 1549,   11, -100, -100, -100, -100, -100])


## Wikitext-2

### Data source from Hugging Face

In [ ]:
dataset = datasets.load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')

In [ ]:
print(len(dataset), type(dataset), dataset)
print(dataset[100])

4358 <class 'datasets.arrow_dataset.Dataset'> Dataset({
    features: ['text'],
    num_rows: 4358
})
{'text': ' Du Fu \'s popularity grew to such an extent that it is as hard to measure his influence as that of Shakespeare in England : it was hard for any Chinese poet not to be influenced by him . While there was never another Du Fu , individual poets followed in the traditions of specific aspects of his work : Bai Juyi \'s concern for the poor , Lu You \'s patriotism , and Mei Yaochen \'s reflections on the quotidian are a few examples . More broadly , Du Fu \'s work in transforming the lǜshi from mere word play into " a vehicle for serious poetic utterance " set the stage for every subsequent writer in the genre . \n'}


In [ ]:
dataset[100]


{'text': ' Du Fu \'s popularity grew to such an extent that it is as hard to measure his influence as that of Shakespeare in England : it was hard for any Chinese poet not to be influenced by him . While there was never another Du Fu , individual poets followed in the traditions of specific aspects of his work : Bai Juyi \'s concern for the poor , Lu You \'s patriotism , and Mei Yaochen \'s reflections on the quotidian are a few examples . More broadly , Du Fu \'s work in transforming the lǜshi from mere word play into " a vehicle for serious poetic utterance " set the stage for every subsequent writer in the genre . \n'}

### Data source from torchtext
https://pytorch.org/tutorials/beginner/transformer_tutorial.html

In [ ]:
train_iter = WikiText2(root='../data/text', split='test')
tokenizer = get_tokenizer('basic_english')


In [ ]:
# vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
# vocab.set_default_index(vocab['<unk>'])
# len(vocab)

In [ ]:
# vocab['the']

In [ ]:
# vocab(tokenizer('this is a test'))

In [ ]:
# # concatenate all sentences together
def data_process(raw_text_iter) -> torch.Tensor:
    """Converts raw text into a flat Tensor."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

In [ ]:
# for idx, i in enumerate(train_iter):
#     print(idx, i)
#     print(vocab(tokenizer(i)))

In [ ]:
# train_iter, val_iter, test_iter = WikiText2()
# train_data = data_process(train_iter)
# val_data = data_process(val_iter)
# test_data = data_process(test_iter)

In [ ]:
def batchify(data: torch.Tensor, bsz: int) -> torch.Tensor:
    """Divides the data into ``bsz`` separate sequences, removing extra elements
    that wouldn't cleanly fit.

    Arguments:
        data: Tensor, shape ``[N]``
        bsz: int, batch size

    Returns:
        Tensor of shape ``[N // bsz, bsz]``
    """
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).t().contiguous()
    return data

In [ ]:
# test_data = batchify(test_data, 10)
# print(test_data)


In [ ]:
bptt = 35
def get_batch(source: torch.Tensor, i: int) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Args:
        source: Tensor, shape ``[full_seq_len, batch_size]``
        i: int

    Returns:
        tuple (data, target), where data has shape ``[seq_len, batch_size]`` and
        target has shape ``[seq_len * batch_size]``
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len]
    return data, target

In [ ]:
# x, y = get_batch(test_data, 0)
# print("x: ", x[:2])
# print("y: ", y[:2])

### Word-based tokenization 

#### torchtext tokenizer

In [ ]:
# tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
# vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
# # vocab.set_default_index(vocab['<unk>'])
# tokenize_data = lambda example, tokenizer: {'tokens': tokenizer(example['text'])}  
# tokenized_dataset = dataset.map(tokenize_data, remove_columns=['text'], 
# fn_kwargs={'tokenizer': tokenizer})
# print(tokenized_dataset['train'][88]['tokens'])

#### hugging face tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
tokens = tokenizer.tokenize(dataset[100]['text'])
print(tokens)

['Du', 'Fu', "'", 's', 'popularity', 'grew', 'to', 'such', 'an', 'extent', 'that', 'it', 'is', 'as', 'hard', 'to', 'measure', 'his', 'influence', 'as', 'that', 'of', 'Shakespeare', 'in', 'England', ':', 'it', 'was', 'hard', 'for', 'any', 'Chinese', 'poet', 'not', 'to', 'be', 'influenced', 'by', 'him', '.', 'While', 'there', 'was', 'never', 'another', 'Du', 'Fu', ',', 'individual', 'poets', 'followed', 'in', 'the', 'traditions', 'of', 'specific', 'aspects', 'of', 'his', 'work', ':', 'Bai', 'Ju', '##yi', "'", 's', 'concern', 'for', 'the', 'poor', ',', 'Lu', 'You', "'", 's', 'pat', '##riot', '##ism', ',', 'and', 'Mei', 'Yao', '##chen', "'", 's', 'reflections', 'on', 'the', 'q', '##uo', '##ti', '##dian', 'are', 'a', 'few', 'examples', '.', 'More', 'broadly', ',', 'Du', 'Fu', "'", 's', 'work', 'in', 'transforming', 'the', '[UNK]', 'from', 'mere', 'word', 'play', 'into', '"', 'a', 'vehicle', 'for', 'serious', 'poetic', 'utter', '##ance', '"', 'set', 'the', 'stage', 'for', 'every', 'subsequen

### Numericalization

#### torchtext

In [ ]:
# vocab = torchtext.vocab.build_vocab_from_iterator(tokenized_dataset['train']['tokens'], 
# min_freq=3) 
# vocab.insert_token('<unk>', 0)           
# vocab.insert_token('<eos>', 1)            
# vocab.set_default_index(vocab['<unk>'])   
# print(len(vocab))                         
# print(vocab.get_itos()[:10])  

#### hugging face

In [ ]:
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)
print(tokenizer.decode(ids))


[12786, 14763, 112, 188, 5587, 2580, 1106, 1216, 1126, 6102, 1115, 1122, 1110, 1112, 1662, 1106, 4929, 1117, 2933, 1112, 1115, 1104, 7647, 1107, 1652, 131, 1122, 1108, 1662, 1111, 1251, 1922, 4225, 1136, 1106, 1129, 4401, 1118, 1140, 119, 1799, 1175, 1108, 1309, 1330, 12786, 14763, 117, 2510, 11587, 1723, 1107, 1103, 7181, 1104, 2747, 5402, 1104, 1117, 1250, 131, 27900, 23915, 10279, 112, 188, 4517, 1111, 1103, 2869, 117, 14557, 1192, 112, 188, 26227, 23326, 1863, 117, 1105, 24563, 27762, 10415, 112, 188, 26906, 1113, 1103, 186, 11848, 3121, 10359, 1132, 170, 1374, 5136, 119, 3046, 14548, 117, 12786, 14763, 112, 188, 1250, 1107, 20892, 1103, 100, 1121, 8574, 1937, 1505, 1154, 107, 170, 3686, 1111, 3021, 15751, 15462, 3923, 107, 1383, 1103, 2016, 1111, 1451, 4194, 2432, 1107, 1103, 6453, 119]
Du Fu's popularity grew to such an extent that it is as hard to measure his influence as that of Shakespeare in England : it was hard for any Chinese poet not to be influenced by him. While there w

## Hugging Face for LM without intermediary steps
https://huggingface.co/course/chapter7/6?fw=pt

In [ ]:
# directly without intermediary steps
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
text = ["this is a text.", "or so it seems"]
padded = tokenizer(text, max_length=4, truncation=True, return_length=True, return_overflowing_tokens=True)

print(padded)
print(padded.keys())
print([tokenizer.decode(x) for x in padded['input_ids']])

{'input_ids': [[101, 1142, 1110, 102], [101, 170, 3087, 102], [101, 119, 102], [101, 1137, 1177, 102], [101, 1122, 3093, 102]], 'token_type_ids': [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]], 'length': [4, 4, 3, 4, 4], 'overflow_to_sample_mapping': [0, 0, 0, 1, 1]}
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'length', 'overflow_to_sample_mapping'])
['[CLS] this is [SEP]', '[CLS] a text [SEP]', '[CLS]. [SEP]', '[CLS] or so [SEP]', '[CLS] it seems [SEP]']


In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
out = data_collator(padded['input_ids'])
for key in out:
    print(f"{key} shape: {out[key].shape}")

print(out['input_ids'], out['labels'])
# Shifting the inputs and labels to align them happens inside the model, so the data collator just copies the inputs to create the labels.

input_ids shape: torch.Size([5, 4])
labels shape: torch.Size([5, 4])
tensor([[ 101, 1142, 1110,  102],
        [ 101,  170, 3087,  102],
        [ 101,  119,  102,    0],
        [ 101, 1137, 1177,  102],
        [ 101, 1122, 3093,  102]]) tensor([[ 101, 1142, 1110,  102],
        [ 101,  170, 3087,  102],
        [ 101,  119,  102, -100],
        [ 101, 1137, 1177,  102],
        [ 101, 1122, 3093,  102]])


### Data loader
Concatenate all data into one large string of text and then chunk it into context length chunks
- https://towardsdatascience.com/language-modeling-with-lstms-in-pytorch-381a26badcbf
- https://www.youtube.com/watch?v=ma1TrR7gE7I&t=273s

In [ ]:
def get_data(dataset, vocab, batch_size):
    data = []                                                   
    for example in dataset:
        if example['tokens']:                                      
            tokens = example['tokens'].append('<eos>')             
            tokens = [vocab[token] for token in example['tokens']] 
            data.extend(tokens)                                    
    data = torch.LongTensor(data)                                 
    num_batches = data.shape[0] // batch_size 
    data = data[:num_batches * batch_size]                       
    data = data.view(batch_size, num_batches)          
    return data

In [ ]:
# batch_size = 1024
# train_data = get_data(tokenized_dataset['train'], vocab, batch_size)
# valid_data = get_data(tokenized_dataset['validation'], vocab, batch_size)
# test_data = get_data(tokenized_dataset['test'], vocab, batch_size)


In [ ]:
# print(tokenized_dataset.shape)
# print(tokenized_dataset['train']['tokens'][88])

## Language modeling dataset

Basically concatenate all data into one big array of ids and then create block_lengths inputs. shift for corresponding labels.

In [ ]:
print(shake)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 32000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 4000
    })
    valid: Dataset({
        features: ['text'],
        num_rows: 4000
    })
})


### Tokenization

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')

In [ ]:
def tokenize_function(examples:List[dict[str,str]]) -> dict[str, List[List[int]]]:
    result = tokenizer(examples["text"]) #, max_length=context_length, truncation=True, return_length=True, return_overflowing_tokens=True)
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

In [ ]:

tokenized = shake.map(
    tokenize_function, batched=True,
    remove_columns=["text"],
    num_proc = 1
)

In [ ]:
print(tokenized['train'], type(tokenized['train']))


Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids'],
    num_rows: 32000
}) <class 'datasets.arrow_dataset.Dataset'>


### Sentences concatenation

In [ ]:
all = []
for k,v in tokenized.items():
    print(k, v)
    for x in v['input_ids']:
        all += x
    print(len(all))
print(all[:15])

train Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids'],
    num_rows: 32000
})
238064
test Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids'],
    num_rows: 4000
})
267859
valid Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids'],
    num_rows: 4000
})
298027
[8496, 674, 826, 46258, 2988, 318, 1716, 13, 1870, 373, 379, 938, 503, 12, 24903]


### Batchify

In [ ]:
def get_batch(data, batch_size, block_size):
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])
    return x, y

In [ ]:
x, y = get_batch(np.array(all), 16, 10)
print(x.shape, y.shape)
print(x[0], y[0])
print(tokenizer.decode(x[0]), tokenizer.decode(y[0]))

torch.Size([16, 10]) torch.Size([16, 10])
tensor([  790, 32460, 33769,   314,   787,   817,   272,   428,  4939,  3211]) tensor([32460, 33769,   314,   787,   817,   272,   428,  4939,  3211,    25])
 every tedious stride I makeThan this weak arm  tedious stride I makeThan this weak arm:


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()